In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))

In [11]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pylab
import statsmodels.api as sm
from pmdarima.arima import auto_arima
import sklearn
import yfinance
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima_model import ARIMA 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats.distributions import chi2
sns.set()

In [12]:
raw_data = yfinance.download(tickers= "^GSPC ^FTSE ^N225 ^GDAXI", start= "1994-01-07", end= "2018-01-29",
                            interval= "1d", group_by='ticker', auto_adjust=True, treads=True)
df_comp = raw_data.copy()

[*********************100%***********************]  4 of 4 completed


In [13]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [14]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225'],df_comp['^GSPC'],df_comp['^GDAXI'],df_comp['^FTSE']
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')

## Creating Returns

In [15]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

## Splitting the data 

In [16]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

## Fitting a Model

In [17]:
model_auto = auto_arima(df.ret_ftse[1:])

In [18]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(3, 0, 5)   Log Likelihood               -7883.727
Date:                Sun, 22 Aug 2021   AIC                          15785.453
Time:                        17:04:22   BIC                          15844.142
Sample:                             0   HQIC                         15806.019
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1274      0.078     -1.626      0.104      -0.281       0.026
ar.L2         -0.8760      0.020    -44.835      0.000      -0.914      -0.838
ar.L3         -0.2776      0.072     -3.879      0.000      -0.418      -0.137
ma.L1          0.1026      0.078      1.311      0.190      -0.051       0.256
ma.L2          0.8223      0.023     36.510      0.000       0.778       0.866
ma.L3          0.1649      0.070      2.362      0.018       0.028       0.302
ma.L4         -0.0085      0.009     -0.916      0.360      -0.027       0.010
ma.L5         -0.1114      0.008    -13.436      0.000      -0.128      -0.095
sigma2         1.3546      0.015     91.104      0.000       1.326       1.384
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6244.58
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.45
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [19]:
model_auto = auto_arima(df.ret_ftse[1:], exogenous = df[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m=5,
                       max_order=None, max_p= 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D=2,
                       maxiter=50, alpha=0.05, n_jobs=-1, trend='ct')


In [25]:
model_auto

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 3, 5),
      suppress_warnings=True, trend='ct', with_intercept=False)

In [26]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         SARIMAX Results                                         
=================================================================================================
Dep. Variable:                                         y   No. Observations:                 5019
Model:             SARIMAX(3, 0, 0)x(0, 0, [1, 2, 3], 5)   Log Likelihood               -5305.714
Date:                                   Sun, 22 Aug 2021   AIC                          10635.428
Time:                                           17:19:37   BIC                          10713.680
Sample:                                       01-10-1994   HQIC                         10662.849
                                            - 04-04-2013                                         
Covariance Type:                                     opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0063      0.019     -0.330      0.741      -0.044       0.031
drift       5.526e-07   6.94e-06      0.080      0.936    -1.3e-05    1.41e-05
ret_spx        0.0854      0.006     13.366      0.000       0.073       0.098
ret_dax        0.5633      0.006    100.515      0.000       0.552       0.574
ret_nikkei     0.0739      0.005     14.923      0.000       0.064       0.084
ar.L1         -0.1199      0.008    -14.245      0.000      -0.136      -0.103
ar.L2         -0.0243      0.011     -2.210      0.027      -0.046      -0.003
ar.L3         -0.0697      0.010     -6.694      0.000      -0.090      -0.049
ma.S.L5       -0.0318      0.010     -3.274      0.001      -0.051      -0.013
ma.S.L10      -0.0559      0.010     -5.394      0.000      -0.076      -0.036
ma.S.L15      -0.0268      0.010     -2.629      0.009      -0.047      -0.007
sigma2         0.4872      0.006     86.427      0.000       0.476       0.498
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):             10037.45
Prob(Q):                              0.92   Prob(JB):                         0.00
Heteroskedasticity (H):               1.04   Skew:                             0.21
Prob(H) (two-sided):                  0.40   Kurtosis:                         9.91
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [21]:
model_auto1 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m=5,
                       max_order=None, max_p= 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D=2,
                       maxiter=50, alpha=0.05, n_jobs=-1, trend='ct', information_criterion='oob',
                       out_of_sample_size= int(len(df_comp)*0.2))

In [24]:
model_auto1

ARIMA(order=(2, 0, 2), out_of_sample_size=1255, scoring_args={},
      seasonal_order=(1, 0, 2, 5), suppress_warnings=True, trend='ct',
      with_intercept=False)

In [22]:
model_auto1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 6275
Model:             SARIMAX(2, 0, 2)x(1, 0, 2, 5)   Log Likelihood               -6357.988
Date:                           Sun, 22 Aug 2021   AIC                          12741.976
Time:                                   17:17:47   BIC                          12829.652
Sample:                                        0   HQIC                         12772.355
                                          - 6275                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0088      0.008     -1.082      0.279      -0.025       0.007
drift       6.202e-07   2.39e-06      0.259      0.795   -4.07e-06    5.31e-06
x1             0.0853      0.006     13.582      0.000       0.073       0.098
x2             0.5634      0.005    102.833      0.000       0.553       0.574
x3             0.0738      0.005     15.656      0.000       0.065       0.083
ar.L1         -0.1405      0.092     -1.522      0.128      -0.321       0.040
ar.L2          0.5456      0.054     10.041      0.000       0.439       0.652
ma.L1          0.0193      0.093      0.208      0.835      -0.163       0.201
ma.L2         -0.5749      0.056    -10.322      0.000      -0.684      -0.466
ar.S.L5       -0.0128      0.187     -0.068      0.946      -0.379       0.353
ma.S.L5       -0.0202      0.187     -0.108      0.914      -0.387       0.347
ma.S.L10      -0.0550      0.013     -4.161      0.000      -0.081      -0.029
sigma2         0.4861      0.005     90.545      0.000       0.476       0.497
===================================================================================
Ljung-Box (L1) (Q):                   2.85   Jarque-Bera (JB):             12788.07
Prob(Q):                              0.09   Prob(JB):                         0.00
Heteroskedasticity (H):               0.55   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                         9.98
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""